In [ ]:
!pip install accelerate
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft
!pip install flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Работа с датасетами (желательно попробовать прикрутить к дз 2)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('nyu-mll/glue', 'cola', split='validation')
# dataset = load_dataset('ai-forever/MERA', 'rcb', split='validation')
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1043
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model = AutoModelForCausalLM.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct', attn_implementation='sdpa', device_map='cuda')
model.eval()

tokenizer = AutoTokenizer.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct')
gen_config = GenerationConfig.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct')
gen_config

config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.34M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.28M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

GenerationConfig {
  "do_sample": true,
  "eos_token_id": 145111,
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [ ]:
gen_config.max_new_tokens = 1
gen_config.do_sample = False

In [ ]:
def sample2messages(sample):
    return [{'role': 'user', 'content': f"Your task is to determine the acceptability of the text for the English language in terms of syntax, morphology and semantics. The answer should be one number: 0 or 1, where 0 means the sentence is not acceptable from the point of view of the English language, 1 means it is acceptable.\nText:{sample['sentence']}"}]

dataset[0], sample2messages(dataset[0])

({'sentence': 'The sailors rode the breeze clear of the rocks.',
  'label': 1,
  'idx': 0},
 [{'role': 'user',
   'content': 'Your task is to determine the acceptability of the text for the English language in terms of syntax, morphology and semantics. The answer should be one number: 0 or 1, where 0 means the sentence is not acceptable from the point of view of the English language, 1 means it is acceptable.\nText:The sailors rode the breeze clear of the rocks.'}])

In [ ]:
tokenizer.apply_chat_template(sample2messages(dataset[0]), add_generation_prompt=True, tokenize=False,return_tensors='pt')

'<|im_start|>user\nYour task is to determine the acceptability of the text for the English language in terms of syntax, morphology and semantics. The answer should be one number: 0 or 1, where 0 means the sentence is not acceptable from the point of view of the English language, 1 means it is acceptable.\nText:The sailors rode the breeze clear of the rocks.<|im_end|>\n<|im_start|>assistant\n'

In [ ]:
import torch
inputs = tokenizer.apply_chat_template(sample2messages(dataset[0]), add_generation_prompt=True, return_tensors='pt')
inputs = inputs.to(model.device)
with torch.no_grad():
    res = model.generate(inputs, generation_config=gen_config).detach().cpu().numpy()[0]
res, tokenizer.decode(res[inputs.shape[1]:])

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


(array([145110,    872,    198,   7771,   3383,    374,    311,   8253,
           279,   4193,   2897,    315,    279,   1467,    369,    279,
          6364,   4128,    304,   3793,    315,  19482,     11,  78512,
           323,  52694,     13,    576,   4226,   1265,    387,    825,
          1372,     25,    220,     15,    476,    220,     16,     11,
          1380,    220,     15,   3363,    279,  11652,    374,    537,
         21555,    504,    279,   1459,    315,   1651,    315,    279,
          6364,   4128,     11,    220,     16,   3363,    432,    374,
         21555,    624,   1178,  74045,  68838,  40661,    279,  45285,
          2797,    315,    279,  23035,     13, 145111,    198, 145110,
         77091,    198,     16]),
 '1')

In [ ]:

def generate(sample):
    inputs = tokenizer.apply_chat_template(sample2messages(sample), add_generation_prompt=True, return_tensors='pt') #shape тут 1 x длина последовательности
    inputs = inputs.to(model.device)
    with torch.no_grad():
        res = model.generate(inputs, generation_config=gen_config).detach().cpu().numpy()[0]

    return tokenizer.decode(res[inputs.shape[1]:]) #только новые токены

In [ ]:
import numpy as np
from tqdm import tqdm

metrics = []
for sample in tqdm(dataset):
    predict = generate(sample)
    metrics.append(predict == str(sample['label']))

print('Accuracy = ', np.mean(metrics))

  0%|          | 0/1043 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
100%|██████████| 1043/1043 [01:44<00:00,  9.98it/s]

Accuracy =  0.7497603068072867


Токенизация примеры

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

model_name_or_path = 'openchat/openchat-3.5-1210'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

In [ ]:
import json
tokenizer_json = json.loads(tokenizer._tokenizer.to_str())
tokenizer_json['model'].keys()

dict_keys(['type', 'dropout', 'unk_token', 'continuing_subword_prefix', 'end_of_word_suffix', 'fuse_unk', 'byte_fallback', 'ignore_merges', 'vocab', 'merges'])

In [ ]:
list(tokenizer_json['model']['vocab'].items())[255:290]


[('<0xFC>', 255),
 ('<0xFD>', 256),
 ('<0xFE>', 257),
 ('<0xFF>', 258),
 ('▁▁', 259),
 ('▁▁▁▁', 260),
 ('▁t', 261),
 ('in', 262),
 ('er', 263),
 ('▁a', 264),
 ('he', 265),
 ('on', 266),
 ('re', 267),
 ('▁s', 268),
 ('en', 269),
 ('at', 270),
 ('or', 271),
 ('▁the', 272),
 ('▁▁▁▁▁▁▁▁', 273),
 ('es', 274),
 ('▁w', 275),
 ('an', 276),
 ('▁c', 277),
 ('is', 278),
 ('it', 279),
 ('ou', 280),
 ('▁d', 281),
 ('al', 282),
 ('ar', 283),
 ('▁p', 284),
 ('▁f', 285),
 ('ed', 286),
 ('▁b', 287),
 ('ing', 288),
 ('▁o', 289)]

In [ ]:
tokenizer_json['model']['merges'][:20]

[['▁', 't'],
 ['i', 'n'],
 ['e', 'r'],
 ['▁', 'a'],
 ['h', 'e'],
 ['o', 'n'],
 ['r', 'e'],
 ['▁', 's'],
 ['e', 'n'],
 ['a', 't'],
 ['o', 'r'],
 ['▁t', 'he'],
 ['▁th', 'e'],
 ['▁', 'the'],
 ['e', 's'],
 ['▁', 'w'],
 ['a', 'n'],
 ['▁', 'c'],
 ['i', 's'],
 ['i', 't']]

In [ ]:
tokenizer('Large 1Large')['input_ids']


[1, 23292, 28705, 28740, 27408]

In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer('Large 1Large')['input_ids'])


['<s>', '▁Large', '▁', '1', 'Large']

In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer('Large 1Large', add_special_tokens=False)['input_ids'])


['▁Large', '▁', '1', 'Large']

In [ ]:
model_name_or_path = 'Qwen/Qwen2.5-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
tokenizer_json = json.loads(tokenizer._tokenizer.to_str())
tokenizer_json['model'].keys()


dict_keys(['type', 'dropout', 'unk_token', 'continuing_subword_prefix', 'end_of_word_suffix', 'fuse_unk', 'byte_fallback', 'ignore_merges', 'vocab', 'merges'])

In [ ]:
list(tokenizer_json['model']['vocab'].items())[250:290]


[('ľ', 250),
 ('Ŀ', 251),
 ('ŀ', 252),
 ('Ł', 253),
 ('ł', 254),
 ('Ń', 255),
 ('ĠĠ', 256),
 ('ĠĠĠĠ', 257),
 ('in', 258),
 ('Ġt', 259),
 ('ĠĠĠĠĠĠĠĠ', 260),
 ('er', 261),
 ('ĠĠĠ', 262),
 ('on', 263),
 ('Ġa', 264),
 ('re', 265),
 ('at', 266),
 ('st', 267),
 ('en', 268),
 ('or', 269),
 ('Ġth', 270),
 ('ĊĊ', 271),
 ('Ġc', 272),
 ('le', 273),
 ('Ġs', 274),
 ('it', 275),
 ('an', 276),
 ('ar', 277),
 ('al', 278),
 ('Ġthe', 279),
 (';Ċ', 280),
 ('Ġp', 281),
 ('Ġf', 282),
 ('ou', 283),
 ('Ġ=', 284),
 ('is', 285),
 ('ĠĠĠĠĠĠĠ', 286),
 ('ing', 287),
 ('es', 288),
 ('Ġw', 289)]

In [ ]:
tokenizer_json['model']['merges'][:20]


[['Ġ', 'Ġ'],
 ['ĠĠ', 'ĠĠ'],
 ['i', 'n'],
 ['Ġ', 't'],
 ['ĠĠĠĠ', 'ĠĠĠĠ'],
 ['e', 'r'],
 ['ĠĠ', 'Ġ'],
 ['o', 'n'],
 ['Ġ', 'a'],
 ['r', 'e'],
 ['a', 't'],
 ['s', 't'],
 ['e', 'n'],
 ['o', 'r'],
 ['Ġt', 'h'],
 ['Ċ', 'Ċ'],
 ['Ġ', 'c'],
 ['l', 'e'],
 ['Ġ', 's'],
 ['i', 't']]

In [ ]:
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode
from typing import Dict, Optional

byte_encoder = bytes_to_unicode()

def token_bytes_to_string(b):
  return ''.join([byte_encoder[ord(char)] for char in b.decode('latin-1')])

token_bytes_to_string(' Пр'.encode())

'ĠÐŁÑĢ'

'ĠÐŁÑĢ'

In [ ]:
tokenizer_json['model']['vocab'][token_bytes_to_string(' Пр'.encode())]


79484

In [ ]:
tokenizer(' Пр')['input_ids']


[79484]

In [ ]:
some_token = token_bytes_to_string(' Пр'.encode())
tokenizer.convert_tokens_to_string([some_token])

' Пр'

In [ ]:
tokenizer.decoder.decode(['ĠÐŁÑĢ'])

' Пр'

In [ ]:
# Не все токены словаря имеют разумное представление в utf-8 кодировке

for token, token_id in list(tokenizer_json['model']['vocab'].items())[:256]:
    print(token, tokenizer.convert_tokens_to_string([token]))

! !
" "
# #
$ $
% %
& &
' '
( (
) )
* *
+ +
, ,
- -
. .
/ /
0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
: :
; ;
< <
= =
> >
? ?
@ @
A A
B B
C C
D D
E E
F F
G G
H H
I I
J J
K K
L L
M M
N N
O O
P P
Q Q
R R
S S
T T
U U
V V
W W
X X
Y Y
Z Z
[ [
\ \
] ]
^ ^
_ _
` `
a a
b b
c c
d d
e e
f f
g g
h h
i i
j j
k k
l l
m m
n n
o o
p p
q q
r r
s s
t t
u u
v v
w w
x x
y y
z z
{ {
| |
} }
~ ~
¡ �
¢ �
£ �
¤ �
¥ �
¦ �
§ �
¨ �
© �
ª �
« �
¬ �
® �
¯ �
° �
± �
² �
³ �
´ �
µ �
¶ �
· �
¸ �
¹ �
º �
» �
¼ �
½ �
¾ �
¿ �
À �
Á �
Â �
Ã �
Ä �
Å �
Æ �
Ç �
È �
É �
Ê �
Ë �
Ì �
Í �
Î �
Ï �
Ð �
Ñ �
Ò �
Ó �
Ô �
Õ �
Ö �
× �
Ø �
Ù �
Ú �
Û �
Ü �
Ý �
Þ �
ß �
à �
á �
â �
ã �
ä �
å �
æ �
ç �
è �
é �
ê �
ë �
ì �
í �
î �
ï �
ð �
ñ �
ò �
ó �
ô �
õ �
ö �
÷ �
ø �
ù �
ú �
û �
ü �
ý �
þ �
ÿ �
Ā  
ā 
Ă 
ă 
Ą 
ą 
Ć 
ć 
Ĉ 
ĉ 	
Ċ 

ċ 
Č 
č 
Ď 
ď 
Đ 
đ 
Ē 
ē 
Ĕ 
ĕ 
Ė 
ė 
Ę 
ę 
Ě 
ě 
Ĝ 
ĝ 
Ğ 
ğ 
Ġ  
ġ 
Ģ �
ģ �
Ĥ �
ĥ �
Ħ �
ħ �
Ĩ �
ĩ �
Ī �
ī �
Ĭ �
ĭ �
Į �
į �
İ �
ı �
Ĳ �
ĳ �
Ĵ �
ĵ �
Ķ �
ķ �
ĸ �
Ĺ �
ĺ �
Ļ �
ļ �
Ľ �
ľ

BERT работа MLM и NSP

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM


model_name = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True

In [ ]:
def predict_masked_sent(text, top_k=5):
    text = f"<s>{text}</s>"
    tokenized_text = tokenizer.tokenize(text)
    print(tokenized_text)
    mask_index = tokenized_text.index("<mask>")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to(model.device)

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0].detach().cpu()

    probs = torch.nn.functional.softmax(predictions[0, mask_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted = True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.decode([pred_idx])
        token_weight = top_k_weights[i]
        print(f"<mask>: '{predicted_token}'","| weights:",float(token_weight))

In [ ]:
# predict_masked_sent("Я пью горячий <mask> по утрам")
predict_masked_sent("Violets are <mask>")
predict_masked_sent("This folwers are roses. I know one little poem. <mask> are red. Violets are blue. Nobody cares of you.")
predict_masked_sent("His lips are blue, this man is <mask> in water")

# ну и говно конечно

['<s>', '▁Violet', 's', '▁are', '<mask>', '</s>']
<mask>: 'beautiful' | weights: 0.13093724846839905
<mask>: 'blue' | weights: 0.056204456835985184
<mask>: 'Beautiful' | weights: 0.036910999566316605
<mask>: 'white' | weights: 0.031309232115745544
<mask>: 'green' | weights: 0.026356367394328117
['<s>', '<mask>', '▁are', '▁red', '</s>']
<mask>: 'They' | weights: 0.3219626545906067
<mask>: 'You' | weights: 0.1333846002817154
<mask>: 'they' | weights: 0.11797267198562622
<mask>: 'We' | weights: 0.07484081387519836
<mask>: 'you' | weights: 0.056975651532411575
['<s>', '▁His', '▁lips', '▁are', '▁blue', ',', '▁this', '▁man', '▁is', '<mask>', '▁in', '▁water', '</s>']
<mask>: 'covered' | weights: 0.1437779814004898
<mask>: 'blue' | weights: 0.07365287095308304
<mask>: 'swimming' | weights: 0.06910184770822525
<mask>: 'white' | weights: 0.05319202318787575
<mask>: 'still' | weights: 0.042144861072301865


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
model_input = tokenizer("Я вышел на улицу.", "Там было пасмурно.", max_length=512, return_tensors="pt",truncation=True)
model_input

{'input_ids': tensor([[  101,   839, 10505,  1469, 23214,   132,   102, 12135,  3332, 28937,
         40816,  1438,   132,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
print(tokenizer.decode(model_input['input_ids'][0].tolist()))

[CLS] Я вышел на улицу. [SEP] Там было пасмурно. [SEP]


In [ ]:
def predict_nsp(text1,text2):
    model_input = tokenizer(text1,text2, max_length=512, return_tensors="pt",truncation=True)
    model_input = {k: v.to(model.device) for k, v in model_input.items()}
    with torch.no_grad():
        logits = model(**model_input).logits.detach().cpu()
        probs = torch.nn.functional.softmax(logits, dim=-1)[0]
    return probs.argmax(), probs

In [ ]:
print(predict_nsp("Я проснулся рано утром, встал с кровати и пошел умываться.", "Первым делом я почистил зубы, а потом поел и пошёл в школу к моему другу"))

(tensor(0), tensor([0.6917, 0.3083]))
